# Refcat Loader Demo

Based on ongoing development by Eli Rykoff to load a reference catalog with color terms applied.

## Setup

Currently the development is done on a branch, so some extra steps are needed right at the moment.

In the terminal, go to directory where you keep code rrepositories and clone the [pipe_tasks](https://github.com/lsst/pipe_tasks) repo.

```git clone https://github.com/lsst/pipe_tasks.git
cd pipe_tasks
git checkout tickets/DM-30054```

In your `~/notebooks/.user_setups` file, add the line

`setup -k -r ~/path_to_your_repos/pipe_tasks`

so that the notebook can access your local version of pipe_tasks package.

In [ ]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
import os.path
import numpy as np
from astropy.time import Time

import lsst.geom
from lsst.pipe.tasks.loadReferenceCatalog import LoadReferenceCatalogConfig, LoadReferenceCatalogTask
from lsst.meas.algorithms import ReferenceObjectLoader
import lsst.daf.butler as dafButler
from lsst.utils import getPackageDir

import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
refDataset = 'gaia_dr2_20200414'
# refDataset = 'ps1_pv3_3pi_20170110'

repo = '/repo/main/'
config = os.path.join(repo, 'butler.yaml')
butler = dafButler.Butler(config=config)
registry = butler.registry
collection = 'refcats'
registry.getCollectionSummary(collection)
visit = 35892
datasetRefs = list(registry.queryDatasets(datasetType=refDataset,
                                          collections=collection,
                                          instrument='HSC',
                                          skymap='hsc_rings_v1',
                                          where=f'visit={visit}').expanded())

dataIds = [_.dataId for _ in datasetRefs]

refCats = [butler.getDeferred(refDataset, _, collections=['refcats'])
           for _ in dataIds]

#cat_ref = butler.getDirect(datasetRefs[0])
cat_ref_example = butler.getDirect(datasetRefs[0])

# Get the HSC catalog for comparsion
refs = list(registry.queryDatasets(datasetType='sourceTable_visit',
                                   collections=['HSC/runs/RC2/w_2021_18/DM-29973'],
                                   instrument='HSC',
                                   skymap='hsc_rings_v1',
                                   where=f'visit={visit}'))
cat_hsc = butler.getDirect(refs[0])

config = LoadReferenceCatalogConfig()
config.refObjLoader.ref_dataset_name = refDataset

# EXTRA
if refDataset == 'gaia_dr2_20200414':
    config.refObjLoader.requireProperMotion = True
    config.refObjLoader.anyFilterMapsToThis = 'phot_g_mean'
    config.doApplyColorTerms = False
else:
    config.refObjLoader.load(os.path.join(getPackageDir('obs_subaru'),
                                          'config',
                                          'filterMap.py'))
    config.colorterms.load(os.path.join(getPackageDir('obs_subaru'),
                                        'config',
                                        'colorterms.py'))

epoch = Time('2021-06-10')

loaderTask = LoadReferenceCatalogTask(config=config,
                                      dataIds=dataIds,
                                      refCats=refCats)
# Define center relative to HSC catalog
center = lsst.geom.SpherePoint(np.median(cat_hsc['coord_ra']),
                               np.median(cat_hsc['coord_dec']),
                               lsst.geom.degrees)
# Define center relative to reference catalog
#center = lsst.geom.SpherePoint(cat_ref_example['coord_ra'][0],
#                               cat_ref_example['coord_dec'][0],
#                               lsst.geom.radians)
print(center)

cat_ref = loaderTask.getSkyCircleCatalog(center,
                                         1.0*lsst.geom.degrees,
                                         ['HSC-G', 'HSC-R'],
                                         epoch=epoch)
print(len(cat_ref))


In [ ]:
cat_ref

In [ ]:
plt.figure()
plt.scatter(cat_ref['ra'], cat_ref['dec'], marker='.', s=10, edgecolor='none')
plt.scatter(cat_hsc['coord_ra'], cat_hsc['coord_dec'], marker='.', s=1, edgecolor='none')
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')

In [ ]:
cat_hsc